# Advent of Code 2020, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202020%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL. Why? A language that doesn't affect the way you think about programming is not worth knowing.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [1]:
⍝ Helper functions and common settings
⎕FR ⎕PP ⎕IO←1287 34 0
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
rg←{⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)} ⍝ capture groups as vector
'segs'⎕CY'dfns'
nums←⍎¨∊∘⎕D⊆⊢

⍝ Some visualisation help, please
]box on -style=max -trains=tree -fns=on
]rows on

done

┌→─────────────────────────────────────┐
│Was ON -style=min -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→──────┐
│Was OFF│
└───────┘

### Day 1: Report Repair
https://adventofcode.com/2020/day/1

Outer product sum, find locations of 2020 and multiply. 

In [50]:
⊢result←1 2{×/⍵[⊃⍸2020=∘.+⍣⍺⍨⍵]}¨⊂⍎⍕⊃⎕NGET'data/2020/day01.txt'1
assert 73371 127642310≡result

┌→──────────────┐
│73371 127642310│
└~──────────────┘

### Day 2: Password Philosophy
https://adventofcode.com/2020/day/2

In [40]:
⎕IO←1
DAY02←⍎¨@1 2¨'- :'∘segs¨⊃⎕NGET'data/2020/day02.txt'1

In [42]:
⊢result←+⌿↑{(b t l s)←⍵⋄((b∘≤∧t∘≥)+/l=s)(⊃(l=b⊃s)≠l=t⊃s)}¨DAY02
assert 528 497≡result

┌→──────┐
│528 497│
└~──────┘

Ok, perhaps a bit messy. Here's the same idea, in long-hand:

In [43]:
]dinput
Part1←{
    (min max letter string)←⍵
    count←+/letter=string
    (min∘≤∧≤∘max) count
}

In [47]:
]dinput
Part2←{
    (min max letter string)←⍵                ⍝ De-structure 
    ⊃(letter=min⊃string)≠(letter=max⊃string) ⍝ Pick the values, and XOR
}

In [45]:
assert 528=+/Part1¨DAY02
assert 497=+/Part2¨DAY02  

Another neat trick worth remembering is that the built-in ⎕CSV can do numeric conversion:

In [68]:
b t l s←↓⍉⎕CSV('\W+'⎕R','⊃⎕NGET'data/2020/day02.txt' 1)''4

Here's another version, jazzed up by ninja-master @ngn:

In [46]:
p q l s←↓⍉↑DAY02
+/(p∘≤∧≤∘q)+/¨s=∊l
+/≠⌿↑p q⊃¨¨⊂s=∊l

⍝ Or as a matrix
b←↑s=∊l
+/(p∘≤∧≤∘q)+/b
+/≠/(p,⍪q)⊃⍤0 1⍤1⊢b

528

497

528

497

### Day 3: Toboggan Trajectory
https://adventofcode.com/2020/day/3

The first approach that comes to mind is a reduce over the path's steps which we can implement either as a tail-recursive function, or by pre-calculating the coordinate vector.

In [61]:
⎕IO←0
DAY03←↑⊃⎕NGET'data/2020/day03.txt'1

In [62]:
]dinput
Slope←{
    (dy dx)←⍵
    d←⍺
    0 { ⍝ Tail-recursive accumulation
        (y x)←⍵
        y≥≢d:⍺
        (⍺+'#'=d[y;(1⊃⍴d)|x])∇dy dx + y x
    } 0 0
}

In [63]:
⊢result←DAY03∘Slope¨(1 1)(1 3)(1 5)(1 7)(2 1)

┌→──────────────┐
│68 203 78 77 40│
└~──────────────┘

In [64]:
assert 203=1⊃result ⍝ Part 1
assert 3316272960=×/result

Considerably more idiomatically, we can simply create the coordinate vector representing the slope first.

In [66]:
]dinput
Slope2←{
    coords←↓⍉↑(⍴⍺)|⍵×⊂⍳⌈(≢⍺)÷⊃⍵
    +/'#'=⍺[coords]
}

In [67]:
⊢result←DAY03∘Slope2¨(1 1)(1 3)(1 5)(1 7)(2 1)

┌→──────────────┐
│68 203 78 77 40│
└~──────────────┘

### Day 4: Passport Processing
https://adventofcode.com/2020/day/4

In [120]:
DAY04←⊃¨{,/¨(' :'∘segs¨⍵)⊆⍨×≢¨⍵} ⊃⎕NGET'data/2020/day04.txt'1
eyr←(2020∘≤∧≤∘2030)⍎
iyr←(2010∘≤∧≤∘2020)⍎
byr←(1920∘≤∧≤∘2002)⍎
ecl←'amb' 'blu' 'brn' 'gry' 'grn' 'hzl' 'oth'∊⍨⊂
pid←≢'^\d{9}$'⎕S''
hcl←≢'^#[a-f0-9]{6}$'⎕S''
cid←{1}
hgt←≢'^(((59|6[0-9]|7[0-6])in)|((1[5-8][0-9]|19[0-3])cm))$'⎕S''

In [121]:
part1←'byr' 'iyr' 'eyr' 'hgt' 'hcl' 'ecl' 'pid'∘(∧/∊)¨pp←(0≠≢¨DAY04)/DAY04
part2←{∧/{(⍎⍺)⍵}/(2÷⍨≢⍵)2⍴⍵}¨part1/pp
⊢result←+/¨part1 part2
assert 256 198≡result

┌→──────┐
│256 198│
└~──────┘

### Day 5: Binary Boarding
https://adventofcode.com/2020/day/5

In [48]:
⊢part1←⌈/seats←2⊥'BR'∊⍨⍉↑⊃⎕NGET'data/2020/day05.txt'1
assert 888=part1

888

In [243]:
⊢part2←1+(⍸~2{⍵=⍺+1}/sorted)⊃sorted←(⍋⌷¨⊂)seats
assert 522=part2

522

### Day 6: Custom Customs
https://adventofcode.com/2020/day/6

In [122]:
DAY06←{⍵⊆⍨×≢¨⍵}⊃⎕NGET'data/2020/day06.txt'1 ⍝ Groups separated by empty

In [123]:
⊢part1←≢∊∪/¨DAY06
assert 6416=part1

6416

In [124]:
⊢part2←≢∊∩/¨DAY06
assert 3050=part2

3050

### Day 7: Handy Haversacks
https://adventofcode.com/2020/day/7

In [78]:
DAY07←⊃⎕NGET'data/2020/day07.txt'1
bags←'^(\w+)\s(\w+)'⎕S'&'⊢DAY07
contained←(⊃,/)¨'(?:\d+|no)\s(\w+\s\w+)'⎕S rg¨DAY07

In [90]:
no←0
counts←(⍎¨∘⊃,/)¨'(\d+|no)'⎕S rg¨DAY07
cap←↓⍉↑(contained)(counts)

In [79]:
]dinput
Part1←{
    ⍺←⍬
    0=≢,⍵:¯1+≢⍺
    item←⊃,⍵
    idx←⍸∨/(⊂item)⍷↑contained
    (⍺,⊂item)∇⍺~⍨bags[idx],1↓⍵
}

In [80]:
⊢part1←Part1 ⊂'shiny gold'
assert 185=part1

185

In [91]:
Part2←{⍵≡'other bags': 0 ⋄ +/{(b c)←⍵⋄c×1+Part2 b}¨↓⍉↑(bags⍳⊂⍵)⊃cap}

In [92]:
⊢part2←Part2 'shiny gold'
assert 89084=part2

89084

### Day 8: Handheld Halting
https://adventofcode.com/2020/day/8

In [34]:
DAY08←⊃⎕NGET'data/2020/day08.txt'1
(op arg)←↓⍉↑' '(≠⊆⊢)¨DAY08
init←{(c a)←⍺ ⍵⋄{⍵⊣⍵.(code args ip acc lines)←c a 0 0(0/⍨≢c)}⎕NS''}
halted←{⍵.ip≥≢⍵.code}
infloop←{2∊⍵.lines}
tweak←{to←⍵⋄{(⊂to)@⍵⊢op}¨⍸(⊂⍺)≡⍤0⊢op}

In [35]:
]dinput
step←{
    instr←⍵.ip⊃⍵.code
    arg←⍵.ip⊃⍵.args
    ⍵.lines[⍵.ip]+←1
    infloop ⍵:⍵
    'nop'≡instr:⍵⊣⍵.ip+←1
    'acc'≡instr:⍵⊣⍵.(acc ip)+←arg 1
    ⍵⊣⍵.ip+←arg
}

In [36]:
run←step⍣{(infloop ⍺)∨halted ⍺}⊢

In [37]:
result←run (op init ⍎¨arg)
⊢part1←result.acc
assert 1134=part1

1134

In [38]:
]dinput
Part2←{
    0=≢⍵:⍬
    result←run (0⊃⍵)init ⍺
    halted result:result
    ⍺∇1↓⍵
}

In [39]:
result←(⍎¨arg)∘Part2 ⊃,/tweak⌿2 2⍴('jmp')('nop')('nop')('jmp')
⊢part2←result.acc
assert 1205=part2

1205

### Day 9: Encoding Error
https://adventofcode.com/2020/day/9

The running sums in part 1 change very little, so can be cached. In retrospect, the performance gain was probably negligible.

In [238]:
DAY09←⍎⍕⊃⎕NGET'data/2020/day09.txt'1

In [239]:
]dinput
Day09←{
    (i s)←⍵
    r←(i-24)+⍳25                  ⍝ Considered range
    d←DAY09[r]+(¯1↑r)⊃DAY09       ⍝ Changed vals
    n←1⌽1⊖s ⋄ n[;24]←d ⋄ n[24;]←d ⍝ Update the sums cache. Also: new←25 25↑1 1↓cache
    (i+1) (n)
}

In [240]:
(i _)←Day09⍣{(i s)←⍺ ⋄ ~s∊⍨i⊃DAY09} 25 (∘.+⍨25↑DAY09)
⊢part1←i⊃DAY09
assert 1639024365=part1

1639024365

In [241]:
win←part1{⍺∊⍵+/DAY09:⍵⋄⍵+1}⍣≡⊢2
i←part1⍳⍨win+/DAY09

In [242]:
⊢part2←(⌈/+⌊/)DAY09[i+⍳win]
assert 219202240=part2

219202240

We found the stretch by trying increasing lengths of windowed reduction. Another variety is to use scan, noting that the sum of a stretch is the sum to the end of the stretch minus the sum to the beginning of the stretch:

In [181]:
(end start)←⊃⍸part1=∘.-⍨+\DAY09
(⌈/+⌊/)(start+1)↓(end+1)↑DAY09

219202240

### Day 10: Adapter Array
https://adventofcode.com/2020/day/10

In [5]:
⎕IO←0
DAY10←(⍋⌷¨⊂)0,d,3+⌈/d←⍎⍕⊃⎕NGET'data/2020/day10.txt'1

In [6]:
⊢part1←×⌿+/↑1 3=⊂-2-/DAY10
assert 2059=part1

2059

From a solution by @ngn, which roughly goes like so:

```python
with open('data/2020/day10.txt') as f:
    DAY10 = sorted(list(map(int, f)))                 # (⍋⌷¨⊂)⍎⍕⊃⎕NGET'data/2020/day10.txt'1

print(reduce(                                         #             /
    lambda acc, w:[*acc[1:], w*sum(acc)],             # {1↓⍵,⍺×+/⍵}
    [int(i in DAY10) for i in range(1, DAY10[-1]+1)], #              (⌽DAY10∊⍨1+⍳⊃⌽DAY10)
    [0, 0, 1]                                         #                                  ,⊂0 0 1
)[2])                                                 # 2⊃⊃
```

In [7]:
⊢part2←2⊃⊃{1↓⍵,⍺×+/⍵}/(⌽DAY10∊⍨1+⍳⊃⌽DAY10),⊂0 0 1
assert 86812553324672=part2

86812553324672

### Day 11: Seating System
https://adventofcode.com/2020/day/11

In [266]:
DAY11←'L'=↑⊃⎕NGET'data/2020/day11.txt'1 ⍝ seats = 1

Apply the rules: 

- if seat is empty, and no occupied seats → seat filled
- if seat is occupied, and 4 or more occupied neighbours → seat vacated 
- else: state unchanged

In [267]:
Apply←{⍺=0:0⋄(1=⍺)∧0=2⊃⍵:2⋄(2=⍺)∧5≤2⊃⍵:1⋄⍺}

In [268]:
T←{⍵[1;1] Apply +/¨0 1 2=⊂∊⍵} ⍝ State transition, part 1

In [269]:
⊢part1←+/2=∊⊢∘T⌺3 3⍣≡DAY11
assert 2344=part1

2344

For part 2 we're no longer considering the direct neighbours, but the number of seats visible along the eight directions of line of sight.

In [97]:
]dinput
LoS←{
    (m pos)←⍺ ⍵
    max←¯1+≢m
    seat←{b←0∊(0∘≤∧≤∘max)⍺⋄b:1⋄0≠m[⊂⍺]}
    ((⊢≡0⌈max⌊⊢)¨coords)/coords←{⍵+⍣seat pos}¨1-1↓4⌽,⍳3 3
}

In [98]:
T2←{⍺[⊂⍵] Apply +/¨0 1 2=⊂⍺[⍺ LoS ⍵]} ⍝ State transition, part 2

As we're no longer operating in blocks we can't use stencil (⌺) this time. Although all the example data are square, the competition data isn't:

In [81]:
⍴DAY11

┌→────┐
│93 97│
└~────┘

The T2 dfn above assumes that the data are square when testing for bounds. We can solve this by simply padding out the data a bit.

In [99]:
⊢part2←+/2=∊{(⍴⍵)⍴⍵∘T2¨,⍳⍴⍵}⍣≡(2/⌈/⍴DAY11)↑DAY11 ⍝ Takes a few seconds...
assert 2076=part2

2076

This one from @ngn takes a radically different and _much_ faster approach by not using stencil, and using a ravel of the source data. When you flatten (`,⍵`) a matrix m with shape `⍴m` its set of coordinates changes from `⍳⍴m` to `⍳×/⍴m`. They become simply the indices of a vector. The mapping between the two can be done with `k←(⍴m)⊥i j` and its inverse `i j←(⍴m)⊤k` where `i j` are a pair of indices in `m`, and `k` is an index in `,m`.

In [272]:
a←↑⊃⎕NGET'data/2020/day11.txt'1

M←×/n←⍴a
a←'.',⍨,a
b←a≠'.'
c←a='#'
d←1-3 3⊤4~⍨⍳9

f←{x←⍺⋄y←⍵⋄+/{b×(0=s+⍵)∨⍵×x>s←+⌿⍵[y]}⍣≡c}

4 f M,⍨(M×~u)+(n⊥h)×u←⌊⌿(¯1↓b)×⍤1⊢u←(0∘≤∧<[0]∘n)h←d∘.+⍤1⊢n⊤⍳M ⍝ Part 1

5 f M,⍨,[⍳2]{(u@v,[¯.5]v@u)⍴⍨M⊣u v←∊¨1 ¯1↓¨¨⊂⌷∘B∘⊂¨⊢∘⊂⌸⍵}⍤1(+⌿⍪-⌿⍪⊢)n⊤B←⍸b ⍝ Part 2

2344

2076

### Day 12: Rain Risk
https://adventofcode.com/2020/day/12

In [153]:
DAY12←{↓⍉↑(1↑¨⍵)(⍎⍕1↓¨⍵)}⊃⎕NGET'data/2020/day12.txt'1

In [154]:
R←{(2↑⍵),⊂(⍺÷90)⊖2⊃⍵} ⋄ L←{⍵R⍨-⍺}
F←{dv←⍺×(⊃2⊃⍵)⊃(0 1)(1 0)(0 ¯1)(¯1 0)⋄(dv+2↑⍵),2⌷⍵}
N←{((2↑⍵)-⍺ 0),2⌷⍵} ⋄ S←{⍵N⍨-⍺}
E←{((2↑⍵)+0 ⍺),2⌷⍵} ⋄ W←{⍵E⍨-⍺}

In [155]:
⊢part1←+/|2↑⊃{(1⊃⍺)(⍎0⊃⍺)⍵}/(⌽DAY12),⊂(0,0,⊂⍳4)
assert 2879=part1

2879

Part 2 introduces a waypoint, which is relative to the ship's location. The NESW functions move the waypoint location (start value: ⊂1 10). Our accumulator now needs to also keep track of the location of the waypoint:

    wpy, wpx, sy, sx
    
We no longer need a heading vector.

The compass functions remain largely the same:

In [162]:
N←{((2↑⍵)-⍺ 0),¯2↑⍵} ⋄ S←{⍵N⍨-⍺}
E←{((2↑⍵)+0 ⍺),¯2↑⍵} ⋄ W←{⍵E⍨-⍺}

The Forward (F) function adds arg×wp to the ship's position.

In [163]:
F←{d←⍺×2↑⍵ ⋄ (2↑⍵),d+¯2↑⍵}

The rotation functions rotate the waypoint relative to the ship -- this corresponds to reflections.

In [175]:
R←{90=⍺:(⊖¯1 1×2↑⍵),¯2↑⍵⋄180=⍺:(¯1 ¯1×2↑⍵),¯2↑⍵⋄(⊖1 ¯1×2↑⍵),¯2↑⍵}
L←{(360-⍺) R ⍵}

In [176]:
⊢part2←+/|¯2↑⊃{(1⊃⍺)(⍎0⊃⍺)⍵}/(⌽DAY12),⊂(¯1 10 0 0)
assert 178986=part2

178986

### Day 13: Shuttle Search
https://adventofcode.com/2020/day/13

[Chinese Remainder Theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) taken from APL Cart https://aplcart.info/?q=chinese#

In [262]:
(ts tt)←⊃⎕NGET'data/2020/day13.txt'1
time←⍎ts
x←0
buses←⍎','⎕R' '⊢tt

In [263]:
first←⊃⍸⍉↑0=buses|⊂time+⍳20

In [264]:
⊢part1←(⊃first)×buses⊃⍨1⊃first
assert 410=part1

410

In [256]:
CRT←{m|⍵+.×⍺(⊣×⊢|∘⊃{0=⍵:1 0 ⋄ (⍵∇⍵|⍺)+.×0 1,⍪1,-⌊⍺÷⍵})¨⍨⍺÷⍨m←×/⍺}

In [265]:
i←buses⍳buses~0
buses←buses~0

⊢part2←¯1+buses CRT buses-i-1
assert 600691418730595=part2

600691418730595

### Day 14: Docking Data
https://adventofcode.com/2020/day/14

In [273]:
DAY14←{('10X'=⊂7↓0⊃⍵)({⍎⍕'\d+'⎕S'&'⊢⍵}¨1↓⍵)}¨({'mask'≡4↑⍵}¨d)⊂d←⊃⎕NGET'data/2020/day14.txt'1

In [224]:
mem←65436/0
_←{(0⊃⍵)∘{mem[0⊃⍵]←2⊥(((36⍴2)⊤1⊃⍵)∨0⊃⍺)∧~1⊃⍺⋄⍬}¨1⊃⍵}¨DAY14

In [225]:
⊢part1←+/mem
assert 13496669152158=part1

13496669152158

For part 2, we apply the masks to the address, rather than the value. This means we can't pre-allocate the memory, as an addressable space of 36 bits is too large.

In [274]:
mem←⍬
val←⍬

In [275]:
SetMem←{i←mem⍳⍵⋄i<≢mem:⍬⊣val[i]←⍺⋄mem,←⍵⋄val,←⍺⋄⍬}

In [276]:
]dinput
Addr←{
    b←((36⍴2)⊤⊃⍵)∨⊃⍺              ⍝ Address in binary, with set-bit mask applied
    bits←⍉(x⍴2)⊤⍳2*x←+/flb←2⊃⍺
    all←(≢bits)(≢b)⍴b
    all[;⍸flb]←bits               ⍝ 'all' now contains all expanded floating bits
    _←(1⊃⍵)∘SetMem¨2⊥¨↓all
    ⍬
}

In [277]:
_←{(0⊃⍵)∘Addr¨1⊃⍵}¨DAY14          ⍝ Around 30 seconds...

In [278]:
⊢part2←+/val
assert 3278997609887=part2

3278997609887

### Day 15: Rambunctious Recitation
https://adventofcode.com/2020/day/15

This is the [Van Eck](https://www.youtube.com/watch?v=etMJxB-igrc) sequence.

Not playing to APL strengths here; part 2 takes around 70s on my machine.

In [315]:
DAY15←18 8 0 5 4 1 20
spoken←⊃¯1↑DAY15
next←1+≢DAY15
cache←1500⌶¯1↓DAY15
vals←1+⍳¯1+≢DAY15

In [316]:
]dinput
day15←{
    (n i)←⍵
    ~n∊cache: 0,i+1⊣vals,←i-1⊣cache,←n
    d←¯1+i-vals[cache⍳n]
    vals[cache⍳n]←i-1
    d,i+1
}

In [317]:
⊢part1←⊃day15⍣(2020-1+≢cache)⊢spoken next
assert 253=part1

253

In [318]:
cache←1500⌶¯1↓DAY15
vals←1+⍳¯1+≢DAY15

In [319]:
⊢part2←⊃day15⍣(3e7-1+≢cache)⊢spoken next ⍝ ~1m10s
assert 13710=part2

13710

### Day 16: Ticket Translation
https://adventofcode.com/2020/day/16

In [8]:
DAY16←{⍵⊆⍨×≢¨⍵}⊃⎕NGET'data/2020/day16.txt'1

In [9]:
(type first second)←↓⍉↑(⊃,/)¨'^([^:]+):\s([^\s]+)\sor\s(.+)$'⎕S rg¨0⊃DAY16
(first second)←⍎¨¨' '@(=∘'-')¨¨first second
ticket←⊃⍎¨1↓1⊃DAY16
valid←∪∊{⍺+⍳1+⍵-⍺}/↑first,second

In [10]:
⊢part1←+/invalid←(~(∊near)∊valid)/∊near←⍎¨1↓2⊃DAY16
assert 29019=part1

29019

Part 2: match the fields. With 20 fields and ~200 valid tickets, it's too much to search. Instead, we look to resolve the rules by checking which fields they are valid for, and substituting. First we must exclude any tickets known to be invalid.

In [11]:
tickets←(~{∨/⍵∊invalid}¨near)/near

In [12]:
]dinput
resolve←{ ⍝ Find row in rulemap which have single 1. Zero corresp col in others
    ⍺←⍬    
    (≢⍺)=≢⍵:⊢/↑⍸⍵
    fld←⍵
    row←⊃⍺~⍨⍸1=+/fld ⍝ r is first unseen row with single 1
    col←⊃⍸row⌷fld
    fld[;col]←0
    fld[row;col]←1
    fld∇⍨⍺,row
}

First we check every rule against every field in every ticket to see which rules can match each field. The rules are basically just the expanded ranges of integers, concatenated together.

In [14]:
expand←{⍺+⍳1+⍵-⍺}
rules←(⊃,/)¨↓⍉↑(expand/↑first)(expand/↑second)

In [18]:
order←resolve ∧⌿↑(↑tickets)∘.∊rules  ⍝ Apply the rules by outer prod

In [41]:
⊢part2←×/ticket[⍸{'dep'≡3↑⍵}¨type[order]]
assert 517827547723=part2

517827547723

In [16]:
∧⌿↑(↑tickets)∘.∊rules

┌→──────────────────────────────────────┐
↓1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 0 0│
│1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0│
│0 0 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 0 0│
│1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0│
│1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0│
│0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0│
│0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0│
│0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 0 0│
│1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1│
│0 1 1 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 0 0│
│0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0│
│1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 0│
│1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1│
│0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0│
│0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0│
│0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0│
│0 0 0 0 0 0 1 0 1 0 1 0 1 1 1 0 1 1 0 0│
│0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 1 0 0│
│0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0│
│0 0 0 1 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0│
└~──────────────────────────────────────┘

### Day 17: Conway Cubes
https://adventofcode.com/2020/day/17

In [249]:
DAY17←'#'=↑⊃⎕NGET'data/2020/day17.txt'1

In [250]:
Pad←{⍵@(1+⍳⍴⍵)⊢0⍴⍨2+⍴⍵} ⍝ https://aplcart.info/?q=surround%20array#

In [254]:
final←({≢⍸⍵}⌺3 3 3∊¨3+0,¨⊢)⍣6⊢Pad⍣6⊢(↑,∘⊂)DAY17

In [255]:
⊢part1←+/∊final
assert 375=part1

375

In [256]:
final←({≢⍸⍵}⌺3 3 3 3∊¨3+0,¨⊢)⍣6⊢Pad⍣6⊢(↑,∘⊂)⍣2⊢DAY17

In [257]:
⊢part2←+/∊final
assert 2192=part2

2192

### Day 18: Operation Order
https://adventofcode.com/2020/day/18

For part 1, the evaluation rules are those of APL, but R-L instead of L-R. So we flip each expression, mirror brackets and change * to × and evaluate.

In [3]:
DAY18←⊃⎕NGET'data/2020/day18.txt'1
p1←{c←'()*'=⊂r←⊖⍵⋄_←{r[⍸⍵⊃c]←⍵⊃')(×'}¨⍳3⋄⍎r}
⊢part1←+/p1¨DAY18
assert 7293529867931=part1

7293529867931

For part 2, now addition has higher precedence than multiplication. We can approach this in a number of ways. One is to bracket the expression fully, according to the new rules, and then have APL evaluate it for us. For example, given

    5+(8×3+9+3×4×3)
    
we need to transform that to:

    5+(8×(3+9+3)×4×3)

We can solve this using some filthy regexing:

- Repeat each existing paranthesis: 
    
    `5+((8×((3+9+3))×4×3)) → '\(' '\)'⎕R'&&' '&&'⊢`
    
    
- Surround each digit with a pair of parentheses.

    `(5)+(((8)×(((3)+(9)+(3)))×(4)×(3))) → '\d'⎕R'(&)'⊢`
    
    
- Remove the internal brackets in plus-groups 

    `(5+((8)×(((3+9+3)))×(4)×(3))) → '\)\+\('⎕R'+'⊢`

In [6]:
]dinput
Part2←{
    string←'×'@(=∘'*')⊢⍵~' '           ⍝ Remove spaces, and change * to ×
    string←'\(' '\)'⎕R'&&' '&&'⊢string ⍝ Double up parentheses
    string←'\d'⎕R'(&)'⊢string          ⍝ Surround digits with parentheses
    string←'\)\+\('⎕R'+'⊢string        ⍝ Remove internal brackets in +-groups
    ⍎string                            ⍝ Eval
}

In [7]:
⊢part2←+/Part2¨DAY18
assert 60807587180737=part2

60807587180737

### Day 19: Monster Messages
https://adventofcode.com/2020/day/19

Regex the regexes. A set of regular expressions. First resolve rule 0.

In [100]:
DAY19←{⍵⊆⍨×≢¨⍵}⊃⎕NGET'data/2020/day19.txt'1

In [101]:
data←(⊃,/)¨'^(\d+):\s(.+)$'⎕S rg¨0⊃DAY19
rules←('"'⎕R''¨⊃¨1↓¨data)[⍋⍎¨⊣/↑data]

In [102]:
]dinput
subs←{ ⍝ strings←⍺
    (from to)←⍵
    patt←{'\s+'⎕R''⊢'\b',⍕⍵,'\b'}¨from
    repl←{'(?:',⍵,')'}¨to  ⍝ Non-capturing brackets.
    ⍺ {0=≢⍵:⍺⋄((patt⊃⍨⊃⍵)⎕R(repl⊃⍨⊃⍵)⊢⍺)∇1↓⍵} ⍳≢patt
}

In [103]:
]dinput
resolve←{ ⍝ rules←⍵
    nodr←SEEN~⍨⍸{~∨/⎕D∊⍵}¨⍵ ⍝ indices of no-digit rules
    SEEN,←nodr
    ⍵ subs (nodr)(⍵[nodr])  ⍝ substitute everywhere
}

In [104]:
SEEN ← ⍬

In [105]:
rr←resolve⍣{~∨/⎕D∊0⊃⍺} rules    ⍝ Resolve until rule 0 has no digits

In [106]:
patt←'\(\?:([ab ]+)\)'⎕R'\1'⍣2⊢0⊃rr ⍝ Remove some unneeded brackets
patt←'\s'⎕R''⊢patt                  ⍝ Kill spaces
patt←'^',patt,'$'                   ⍝ Anchors

In [107]:
⊢part1←+/{0≠≢⍵}¨patt⎕S'&'¨1⊃DAY19
assert 230=part1

230

Suddenly, the earth moves, and two rules become recursive!

Rules 8 and 11 which used to look like:

    8:  42
    11: 42 31

instead now look like 	

    8:  42 | 42 8
    11: 42 31 | 42 11 31
    
We can do a little bit of manual rewriting here. Looking at 8, it essentially matches one or more 42, so that can be replaced by 

    8: 42+
    
Rule number 11 matches a sequence of 42, followed by the same number of 31, so that can become something like

    11: 42{n} 31{n}


These two rules contribute only to rule zero:

    0: 8 11
    
If we expand that we get 

    0: 42+ 42{n} 31{n}
    0: 42{n+1,} 31{n}
    
..but that still gives us a free parameter. We could guess what the max might be and generate a long list, but we can instead try a slightly broader match: 

    0: ((42)+)((31+))
    
under the provision that we check that the first pattern matches more than the second group. 

In [35]:
]dinput
Part2←{ 
    ⍝ Check that the first group matched more times than the second.
    ⍝ This is technically still too broad -- should be strictly n+1.
    match←⊃⍺⎕S rg⊣⍵
    0=≢match:0
    (m1 m2 m3 m4)←≢¨match
    (m1÷m2)>m3÷m4
}

In [38]:
patt←'^((',(42⊃rr), ')+)((', (31⊃rr), ')+)$'
patt←'\s'⎕R''⊢patt
⊢part2←+/patt∘Part2¨1⊃DAY19
assert 341=part2

341

### Day 20: Jurassic Jigsaw
https://adventofcode.com/2020/day/20

Part 1 can be done without solving the whole board by looking at the edge ids -- the corner pieces have two edges that don't match any other tiles. However, for part 2, we'd still need the full solution, although knowing the corners, the BFS would have a smaller search space. But the full solution is curiously fast in APL without any such cleverness.

Python solution [here](https://gist.github.com/xpqz/23523efeca6c256761367d6131e26262). 

In [262]:
DAY20←{⍵⊆⍨×≢¨⍵}⊃⎕NGET'data/2020/day20.txt'1
TILEIDS←⍎¨⊃,/'(\d+)'⎕S rg ⊃,/{0⊃⍵}¨DAY20
DAY20←↑¨1∘↓¨DAY20
tr←¯1 ¯1∘↓1 1∘↓ ⍝ trim edges of matrix

In [263]:
symm←{(⍵)(⍉∘⊖⍵)(⌽∘⊖⍵)(⍉∘⌽⍵)(⊖⍵)(⍉⍵)(⌽⍵)(⍉∘⌽∘⊖⍵)}
edges←{t←⍉⍵⋄(2⊥'#'=0⌷⍵),(2⊥'#'=⊃↓¯1↑t),(2⊥'#'=⊃↓¯1↑⍵),(2⊥'#'=0⌷t)} ⍝ NESW
gen←{e←edges¨s←symm ⍵⋄(tr¨s)e} ⍝ Build all symmetries, edges and trim.

In [264]:
all←↑gen¨DAY20 
TILES←⊃,/⊣/all
EDGEIDS←⍉↑⊃,/⊢/all
SIZE←12 12

In [265]:
state←{⍵⊣⍵.(next used tiles found)←(0 0) (,⍬) (SIZE⍴¯1) 0}⎕NS''

In [266]:
]dinput
neigh←{
    next←SIZE⊤1+cur←SIZE⊥⍵.next ⍝ Map 2D→1D coordinates, add 1, map back to 2D
    tiles←,⍵.tiles ⋄ used←⍵.used
    
    genStates←{ ⍝ Generate valid new states
        0=≢⍵:⍺
        m←0⊃⍵
        fam←used~⍨⌊m÷8
        ⍬≡fam:⍺ ∇ 1↓⍵
        state←{⍵⊣⍵.(next used tiles found)←next(used,⊃fam)(SIZE⍴m@cur⊢tiles)(0 0≡next)}⎕NS''
        (⍺,state)∇ 1↓⍵
    }

    NW←{ ⍝ Return N and W edge ids
        n←{0<0⊃⍵.next:⍵.tiles[⊂¯1 0+⍵.next]⊃0⌷EDGEIDS ⋄ ⍬}⍵
        w←{0<1⊃⍵.next:⍵.tiles[⊂0 ¯1+⍵.next]⊃3⌷EDGEIDS ⋄ ⍬}⍵
        n w
    }

    (N W)←NW ⍵

    ⍝ Find tiles matching the edge specification.
    ns←{⍵≢⍬:⍵=2⌷EDGEIDS ⋄ 1/⍨1⊃⍴EDGEIDS}N
    ew←{⍵≢⍬:⍵=1⌷EDGEIDS ⋄ 1/⍨1⊃⍴EDGEIDS}W
    ⍬ genStates⍸∧⌿↑(ns)(ew)
}

In [267]:
final←{0=≢⍵:⍬⋄s←⊃1↑⍵⋄s.found:s⋄∇(1↓⍵),neigh ⊃1↑⍵},state ⍝ Breadth-first search

In [268]:
⊢part1←×/TILEIDS[⌊8÷⍨,{↑(⊣/⍵)(⊢/⍵)}⍣2⊢final.tiles] ⍝ Multiply tile-ids of the corners
assert 59187348943703=part1

59187348943703

### Day 21: Allergen Assessment
https://adventofcode.com/2020/day/21

In [136]:
DAY21←', '∘((~∊⍨)⊆⊢)¨¨(⊃,/)¨'^(.+)\s\(contains\s(.+)\)$'⎕S rg¨⊃⎕NGET'data/2020/day21.txt'1
(ingr alg)←∪⌿↑DAY21
recipes←{ingr⍳⍵}¨⊣/↑DAY21
allergens←↓alg∘.{⍵∊⍨∘⊂⍺}⊢/↑DAY21
a2i←(⊃,/)¨recipes∘{∩/⍺[⍸⍵]}¨allergens

In [137]:
Resolve←{⍬≡i←⍸⍺∘{1=≢⍵~⍺}¨⍵:⍺⋄((⊃⍺~⍨(⊃i)⊃⍵)@(⊃i)⊢⍺)∇⍵}

In [138]:
resolved←(¯1/⍨≢a2i) Resolve a2i

We now know which ingredients map to which allergens. For part 1 we're asked to count the number of times the inert (non-allergenic) ingredients occur in the recipes.

In [140]:
⊢part1←≢resolved~⍨∊recipes
assert 2282=part1

2282

For part 2 we seek to arrange the allergenic ingredients alphabetically by their allergen.

In [153]:
⊢part2←1↓','{⊃,/⍺∘,¨⍵}(ingr[resolved])[⍋alg]
assert 'vrzkz,zjsh,hphcb,mbdksj,vzzxl,ctmzsr,rkzqs,zmhnj'≡part2

┌→───────────────────────────────────────────────┐
│vrzkz,zjsh,hphcb,mbdksj,vzzxl,ctmzsr,rkzqs,zmhnj│
└────────────────────────────────────────────────┘

The Resolve function above simply looks through the a2i array finding items of length 1, and substituting those into the  others in turn. We can write that out in more verbose form as:

In [154]:
]dinput
Resolve←{
    ⍺←¯1/⍨≢⍵
    res←⍺
    idx←⍸{1=≢⍵~res}¨⍵        ⍝ First unseen position with 1 element
    ⍬≡idx:res                ⍝ None found; must be done
    res[⊃idx]←⊃res~⍨(⊃idx)⊃⍵ ⍝ Mark as resolved
    res∇⍵                    ⍝ Keep looking
}

### Day 25: Combo Breaker
https://adventofcode.com/2020/day/25

This could be speeded up by noting that this is an example of where the [Baby-Step-Giant-Step](https://en.wikipedia.org/wiki/Baby-step_giant-step) algorithm could be employed. Although it's reasonably swift as brute force.

In [123]:
card_pk←12092626
door_pk←4707356
card_loop←0⋄_←(20201227∘|7∘×)⍣{card_loop+←1⋄⍺=card_pk}⊢1
⊢part1←(20201227∘|door_pk∘×)⍣card_loop⊢1
assert 18329280=part1

18329280